# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

In [2]:
bag = db.read_text("./data/reviews_*.json").map(json.loads)
bag.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [4]:
def get_rating_data(data):
    data, path = data
    filename = re.split("/", path)[-1]
    key = re.findall(r"\d", filename)[0]
    js = json.loads(data)
    js["rating"] = int(key)  
    return js

bag = db.read_text("./data/reviews_*.json", include_path=True).map(get_rating_data)
bag.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3. Посчитайте количество отзывов в исходном датасете.

In [81]:
bag.count().compute()
# 9057540

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [5]:
def by_years(data):
    y = data["date"].split("-")[0]
    if y in ["2014", "2015"]:
        return data

nb = bag.filter(by_years)
# print(nb.take(5))
nb.pluck('date').take(20)

('2014-10-03',
 '2015-05-08',
 '2015-06-30',
 '2015-11-29',
 '2015-08-17',
 '2015-02-02',
 '2015-03-13',
 '2014-03-06',
 '2014-03-26',
 '2014-07-09',
 '2014-10-05',
 '2014-11-19',
 '2014-08-09',
 '2014-03-13',
 '2015-03-01',
 '2015-04-28',
 '2015-08-23',
 '2014-12-26',
 '2014-01-20',
 '2015-03-31')

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [6]:
def proc(data):
    data["review"] = " ".join(re.sub(r'[^a-z ]+', '', data["review"].lower().strip()).split())
    return data

proc_bag = nb.map(proc)
proc_bag.take(3)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time welltoasted bread is the key really toast it i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor i used sourdough bread medium cheddar cheese fast fun great idea made for hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic added the olives under the cheese and baked uncovered for the minutes served with pico de gallo sour cream and avocado chunks fantastic thanks for sharing l

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [77]:
proc_bag.count().compute()
# 735274

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

In [79]:
proc_bag.pluck("rating").frequencies().compute()
# [(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

In [80]:
proc_bag.pluck("rating").mean().compute()
# 4.388036296673077

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [85]:
def find_max(total, cur):
    return max(total, cur, key=lambda x: len(x["review"]))

def find_len(data):
    return {data[0]: len(data[1]["review"])}

proc_bag.foldby("rating", find_max).map(find_len).compute()
# [{0: 6533}, {1: 2854}, {2: 2804}, {3: 3145}, {4: 6533}, {5: 5267}]

[{0: 6533}, {1: 2854}, {2: 2804}, {3: 3145}, {4: 6533}, {5: 5267}]